In [1]:
import json
import pandas as pd
import numpy as np
import xnetwork as xnet
from tqdm.auto import tqdm
from collections import Counter
from itertools import combinations
from collections import defaultdict, namedtuple
from multiprocessing import Pool

In [35]:
def get_impact_by_journal(chu_journal):

    dyear = 1
    papers_per_comm = json.loads(open('bardo_paper_per_year_310522.json').read()) #'310522.json').read())
    papers_per_comm = papers_per_comm[chu_journal.lower()+'_']
    papers_per_year = defaultdict(lambda:[])
    for a,b in papers_per_comm.items():
        for year, listyear in b.items():
            papers_per_year[year] += listyear

    papers = pd.read_csv('paper_journals_cat_170122.csv', sep='\t', header=None)
    papers.columns = ['journal', 'id', 'year', 'categories', 'refs']
    chu_papers = papers[papers.journal == chu_journal]
    chu_papers = set(chu_papers['id'].tolist())

    #     net = xnet.xnet2igraph("subset_chu/citation_net_%s_180122_bardo.xnet" % chu_journal)
    #     net = xnet.xnet2igraph("subset_chu/citation_net_%s_voting_schema_261022.xnet" % chu_journal)

    net = xnet.xnet2igraph("subset_chu/citation_net_nat. mater._011122_voting_schema_bardo_infomap_01112022.xnet")

    all_papers_citing_journal = pd.read_csv('papers_citing_journal_%s.csv' % chu_journal, sep='\t', header=None)
    all_papers_citing_journal.columns = ['id', 'journal', 'year', 'categories', 'refs']
    all_papers_citing_journal = all_papers_citing_journal.dropna()
    

    bardo_dist = defaultdict(lambda: 0.0)
    for y in range(1995, 2020):
    #         subset_papers = all_papers_citing_journal[(all_papers_citing_journal.year >= y) & (all_papers_citing_journal.year <= y+dyear)]
        subset_papers = all_papers_citing_journal[all_papers_citing_journal.year == y + dyear + 1]
        all_refs = []
        print(y)
        for _, paper in subset_papers.iterrows():
            refs = paper['refs'].split(';')
            refs = [r.replace(':', '') for r in refs]
            all_refs += refs

        unique, count = np.unique(all_refs, return_counts=True)
        count_cit = dict()
        for u, c in zip(unique, count):
            count_cit[u] = int(c)
        cited_papers = net.vs.select(wos_id_in = set(all_refs))
        for vtx_paper in cited_papers:
            if y <= vtx_paper['year'] <= y + dyear:
                bardo_dist[y + dyear + 1] += count_cit[vtx_paper['wos_id']]

    print(bardo_dist.keys())

    out_copy = dict()
    C_others = defaultdict(lambda:0)
    P_others = defaultdict(lambda:0)
    for y, count_cit in bardo_dist.items():
        print(y)
        y1 = 0
        y2 = 0
        if str(y-1) in bardo_dist:
            print('aqui')
            y1 = len(set(papers_per_year[str(y-1)]))
        if str(y-2) in papers_per_year:
            y2 = len(set(papers_per_year[str(y-2)]))

#         C_others[y] += count_cit
#         P_others[y] += (y1 + y2)
    
        print(len(bla), y1 + y2)

        if y1 + y2 > 0:
            out_copy[y] = count_cit/(y1 + y2)
        else:
            out_copy[y] = 0

    json_str = json.dumps(out_copy)
    out = open('impact_factor_dist_%s_010622_IFv2_by_journal_260323.json' % chu_journal, 'w')
    out.write(json_str)
    out.close()

In [ ]:
chu_journal = 'ACS Appl. Mater. Interfaces'
journals = [
chu_journal, 'ACS Nano', 'Nano Lett.', 'Nat. Nanotechnol.', 
#         'Nat. Mater.', 
    'Langmuir', 
    "J. Mater. Chem. C", "J. Mater. Chem. A", 
        "J. Mater. Chem.", 
        'Adv. Funct. Mater.', 'Adv. Mater.',
        'J. Mat. Chem. B', 'Chem. Mat.', 
        'J. Am. Chem. Soc.',
        'Macromolecules'
]

# pool = Pool(10)
# pool.map(get_impact, journals)

for j in journals:
    print(j)
    get_impact_by_journal(j)
    # tornar paralelo

In [2]:
file = 'papers_citing_journal_adv. funct. mater..csv'
adv_funct_mat_cits = pd.read_csv(file, on_bad_lines='skip', sep='\t', header=None)

In [3]:
adv_funct_mat_cits.head()

,0,1,2,3,4
0,WOS:000168695800009,Adv. Funct. Mater.,2001,"Chemistry, Multidisciplinary;Chemistry, Physic...",WOS:000085141100010;WOS:A1997WY79200040;WOS:00...
1,WOS:000172967000012,Synth. Met.,2001,"Materials Science, Multidisciplinary;Physics, ...",WOS:000072499100041;WOS:000072913500001;WOS:A1...
2,WOS:000371669000008,Express Polym. Lett.,2016,Polymer Science,WOS:000251267700026;WOS:000235057200004;WOS:00...
3,WOS:000373614400006,J. Alloy. Compd.,2016,"Chemistry, Physical;Materials Science, Multidi...",WOS:000086308400023;WOS:000358852600001;WOS:00...
4,WOS:000387210300008,CARBON NANOSTRUCT,2016,Nanoscience & Nanotechnology;Materials Science...,WOS:000320481900040;WOS:000243683800010;WOS:00...


In [ ]:
# selecionar os artigos publicados em 2016 e pegar as referencias desse ano em particular

In [4]:
cits = defaultdict(lambda:defaultdict(lambda:0))
for idx,row in adv_funct_mat_cits.iterrows():
    for wos_id in row[4].split(';'):
        cits[row[2]][wos_id] += 1

In [5]:
papers = pd.read_csv('paper_journals_cat_170122.csv', sep='\t', header=None)
papers.columns = ['journal', 'id', 'year', 'categories', 'refs']
print(set(papers['journal']))

{'ACS Nano', 'Langmuir', 'Nat. Nanotechnol.', 'Nat. Mater.', 'Macromolecules', 'ACS Appl. Mater. Interfaces', 'J. Mater. Chem. A', 'Nano Lett.', 'J. Mater. Chem. C', 'J. Am. Chem. Soc.', 'Adv. Funct. Mater.', 'J. Mater. Chem.', 'Adv. Mater.', 'Chem. Mat.', 'J. Mat. Chem. B'}


In [6]:
chu_journal = 'Adv. Funct. Mater.'
chu_papers_by_year = papers[papers.journal == chu_journal].groupby('year').apply(lambda x: x['id'])
chu_papers_by_year

year        
2001  335956    WOS:000168695800009
      337196    WOS:000169315500010
      337197    WOS:000171468500006
      337198    WOS:000170368300004
      337420    WOS:000172678700005
                       ...         
2020  348728    WOS:000496529600001
      348729    WOS:000504699300001
      348731    WOS:000495562300001
      348776    WOS:000483939300001
      348789    WOS:000493832300001
Name: id, Length: 12974, dtype: object

In [105]:
import json
import igraph as ig
import WOSRaw as wos
import xnetwork as xnet
from collections import defaultdict

acm_ami = 'ACS Appl. Mater. Interfaces'
adv_f_mater = 'Adv. Funct. Mater.'

valid_journals = [
    'acs nano',
    'adv. mater.',
    'chem. mat.', 
    'j. am. chem. soc.', 
    'j. mat. chem. b', 
    'j. mater. chem. a', 
    'j. mater. chem. c', 
    'langmuir',
    'macromolecules',
    'nano lett.',
    'nat. mater.',
    'nat. nanotechnol.'
]


def get_citnet(current_journal):
    
    papers_by_journal = defaultdict(lambda:[])
    names = []
    titles = []
    years = []
    edges = []
    file = open('toy_chemb_chema_natmat.txt').readlines()
    for line in file:
        entry = json.loads(line)
        item_id = entry['UID']
        item_data = {'UID': item_id, 'data': entry}
        sourcesData = wos.utilities.getSources(item_data)
        journal = wos.utilities.getSourceISOAbbreviation(sourcesData).lower()
        
        publicationInfo = wos.utilities.getPublicationInfo(item_data)
        year = wos.utilities.getPublicationYear(publicationInfo)
        year = int(year)
        
        if journal != current_journal:
            continue
        if year > 2020:
            continue
            
        title = wos.utilities.getTitle(item_data)
        names.append(item_id)
        titles.append(title)
        years.append(int(year))
        
            
        referencesData = wos.utilities.getReferences(item_data)
        ref_uids = wos.utilities.getReferencesUIDs(referencesData)
        for current_uid in ref_uids:
            edges.append((item_id, current_uid))
        
    valid_edges = []
    names_set = set(names)
    for edge in edges:
        if edge[0] in  names_set and edge[1] in names_set:
            valid_edges.append(edge)
    
    
    g = ig.Graph(directed=True)
    g.add_vertices(len(names))
    g.vs['name'] = names
    g.vs['title'] = titles
    g.vs['year'] = years
    g.add_edges(valid_edges)
    return g

for journal in valid_journals:
    print(journal)
    g = get_citnet(journal)
    xnet.igraph2xnet(g, '%s_completedata_170423_allbut.xnet' % journal)

j. mat. chem. b
j. mater. chem. a
nat. mater.


In [68]:
import pandas as pd
current_journal = 'adv. funct. mater.'
all_papers_citing_journal = pd.read_csv('papers_citing_journal_%s.csv' % current_journal, sep='\t', header=None)

In [69]:
net = xnet.xnet2igraph("subset_chu/citation_net_%s_011122_voting_schema_bardo_infomap_01112022.xnet" % current_journal)

In [84]:
import glob
from functools import partial
from multiprocessing import Poo
def get_papers_year(valid_uids, filename):
    papers_by_year = dict()
    with open(filename) as file:
        for line in file:
            paper = json.loads(line)
            referencesData = wos.utilities.getReferences(paper)
            ref_uids = wos.utilities.getReferencesUIDs(referencesData)
            for ref in ref_uids:
                if ref in valid_uids:
                    publicationInfo = wos.utilities.getPublicationInfo(paper)
                    year = wos.utilities.getPublicationYear(publicationInfo)
                    year = int(year)
                    if ref in papers_by_year:
                        if year in papers_by_year[ref]:
                            papers_by_year[ref][year] += 1
                        else:
                            papers_by_year[ref][year] = 1
                    else:
                        papers_by_year[ref] = dict()
                        papers_by_year[ref][year] = 1
    return papers_by_year


def get_papers_year_union(valid_uids):
    files = glob.glob('citing_temp/citing*')
    print(files)
    
    pool = Pool(6)
    result = pool.map(partial(get_papers_year, valid_uids), files)
    
    print('join result...')
    join_result = result[0]
    for r in result[1:]:
        for paper, cit_by_year in r.items():
            for y,c in cit_by_year.items():
                if paper in join_result:
                    if y in join_result[paper]:
                        join_result[paper][y] += c
                    else:
                        join_result[paper][y] = c
                else:
                    join_result[paper] = dict()
                    join_result[paper][y] = c
    
    return join_result


print('get_impact...')
chu_journal = 'adv. funct. mater.'
file = '%s_completedata_110423_bardo_infomap_110423_voting_schema_top15xnet' % chu_journal
net = xnet.xnet2igraph(file)

papers_per_year = defaultdict(lambda:defaultdict(lambda:0))
for vtx in net.vs:
    papers_per_year[vtx['cluster_top15']][vtx['year']] += 1

valid_uids = net.vs['wos_id']
valid_uids = set(valid_uids)

dyear = 1

papers_by_year = get_papers_year_union(valid_uids)

get_impact...
['citing_temp/citing_acm_afm_00', 'citing_temp/citing_acm_afm_01', 'citing_temp/citing_acm_afm_02', 'citing_temp/citing_acm_afm_03', 'citing_temp/citing_acm_afm_04', 'citing_temp/citing_acm_afm_05', 'citing_temp/citing_acm_afm_06', 'citing_temp/citing_acm_afm_07', 'citing_temp/citing_acm_afm_08', 'citing_temp/citing_acm_afm_09']
join result...
cits per year...


In [94]:
def get_impact(chu_journal): # citations
    
    print('get_impact...')
    
    file = '%s_completedata_110423_bardo_infomap_110423_voting_schema.xnet' % chu_journal
    net = xnet.xnet2igraph(file)

    papers_per_year = defaultdict(lambda:0)
    for vtx in net.vs:
        papers_per_year[vtx['year']] += 1
    
    valid_uids = net.vs['wos_id']
    valid_uids = set(valid_uids)
    
    dyear = 1
    
    papers_by_year = get_papers_year_union(valid_uids)
    bardo_dist = defaultdict(lambda: 0.0)


    print('cits per year...')
    for y in range(2010, 2021): # 2000
        
        for paper in net.vs.select(year_le=y-1, year_ge=y-2):
            if paper['wos_id'] in papers_by_year:
                cits_by_year = papers_by_year[paper['wos_id']]

                if y in cits_by_year:
                    bardo_dist[y] += cits_by_year[y]
    
    print(bardo_dist.keys())
    out_copy = dict()
    C_others = defaultdict(lambda:0)
    P_others = defaultdict(lambda:0)
    for y, count_cit in bardo_dist.items():
        y1 = 0
        y2 = 0
        if y-1 in papers_per_year:
            y1 = papers_per_year[y-1]
        if y-2 in papers_per_year:
            y2 = papers_per_year[y-2]

        if y1 + y2 > 0:
            out_copy[y] = count_cit/(y1 + y2)
        else:
            out_copy[y] = 0

    json_str = json.dumps(out_copy)
    out = open('impact_factor_dist_journal_%s.json' % chu_journal, 'w')
    out.write(json_str)
    out.close()

get_impact('Adv. Funct. Mater.')

get_impact...
['citing_temp/citing_acm_afm_00', 'citing_temp/citing_acm_afm_01', 'citing_temp/citing_acm_afm_02', 'citing_temp/citing_acm_afm_03', 'citing_temp/citing_acm_afm_04', 'citing_temp/citing_acm_afm_05', 'citing_temp/citing_acm_afm_06', 'citing_temp/citing_acm_afm_07', 'citing_temp/citing_acm_afm_08', 'citing_temp/citing_acm_afm_09']
join result...
cits per year...
dict_keys([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])


In [95]:
get_impact('ACS Appl. Mater. Interfaces')

get_impact...
['citing_temp/citing_acm_afm_00', 'citing_temp/citing_acm_afm_01', 'citing_temp/citing_acm_afm_02', 'citing_temp/citing_acm_afm_03', 'citing_temp/citing_acm_afm_04', 'citing_temp/citing_acm_afm_05', 'citing_temp/citing_acm_afm_06', 'citing_temp/citing_acm_afm_07', 'citing_temp/citing_acm_afm_08', 'citing_temp/citing_acm_afm_09']
join result...
cits per year...
dict_keys([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])
